In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# name = 'Salesforce/xgen-7b-8k-inst'


tokenizer = AutoTokenizer.from_pretrained("Salesforce/xgen-7b-8k-inst",
                                          trust_remote_code=True)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [3]:
device_map = {"cuda:0" if torch.cuda.is_available() else "cpu"}
device_map

{'cuda:0'}

In [4]:
model = AutoModelForCausalLM.from_pretrained("Salesforce/xgen-7b-8k-inst",
                                             torch_dtype=torch.float16,
                                            #load_in_8bit=True,
                                             device_map='auto',offload_folder='C:/gazala_codes/XGEN_LLM/g_llm/')
                                        


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model.config

LlamaConfig {
  "_name_or_path": "Salesforce/xgen-7b-8k-inst",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0.dev0",
  "use_cache": true,
  "vocab_size": 51200
}

In [6]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.31.0.dev0"
}

In [7]:
from transformers import GenerationConfig
generation_config = model.generation_config
generation_config.max_new_tokens = 128
generation_config.pad_token_id = tokenizer.eos_token_id

In [8]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "max_new_tokens": 128,
  "pad_token_id": 50256,
  "transformers_version": "4.31.0.dev0"
}

In [9]:
system_prompt = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions.\n\n"
)
prompt = f"### Human: What is life?Answer in 1 sentence\n###"

In [10]:
print(system_prompt + prompt)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: What is life?Answer in 1 sentence
###


In [11]:
%%time
inputs = tokenizer(system_prompt + prompt,return_tensors="pt").to(model.device)
#with torch.inference_model():
generations =model.generate(**inputs,generation_config=generation_config)

CPU times: total: 3min 19s
Wall time: 2min 36s


In [12]:
generations

tensor([[   32,  8537,  1022,   257, 11040,  1692,   290,   281, 11666,  4430,
          8796,    13,   383,  8796,  3607,  7613,    11,  6496,    11,   290,
         23507,  7429,   284,   262,  1692,   338,  2683,    13,   198,   198,
         21017,  5524,    25,  1867,   318,  1204,    30, 33706,   287,   352,
          6827,   198, 21017, 15286,    25,  5155,   318,   257,  3716,   290,
          8925,  1080,   326,   318, 16264,   416,  3349,    11, 20728,    11,
           290, 16711,    13,   198, 50256]], device='cuda:0')

In [13]:
print(tokenizer.decode(generations[0], skip_special_tokens=True))

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: What is life?Answer in 1 sentence
### Assistant: Life is a complex and dynamic system that is characterized by growth, reproduction, and adaptation.
<|endoftext|>


# Prompt Helpers


In [36]:
def format_prompt(prompt:str,system_prompt:str)->str:
    return f" "
{system_prompt}
### Human:{prompt}
###
"".strip()

''

In [15]:
system_prompt = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions.\n\n"
)
prompt = f"### Human: What is life?Answer in 1 sentence\n###"
print(prompt,system_prompt)

### Human: What is life?Answer in 1 sentence
### A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.




In [16]:
import re
import textwrap



In [28]:
def clean_response(response:str, wrap_text: bool = True)-> str:
    assistant_prompt = "Assistant:"
    assistant_start = response.find(assistant_prompt)+len(assistant_prompt)
    response = response[assistant_start:response.find("<|endoftext|>")].strip()
    if not wrap_text:
        return response
    return textwrap.fill(response,width=110)

In [29]:
response = tokenizer.decode(generations[0],skip_special_tokens=True)
print(clean_response(response))

Life is a complex and dynamic system that is characterized by growth, reproduction, and adaptation.
